# Segmenting & Clustering Neighborhoods in Toronto

#### Step 1: Import libraries

In [4]:
# import libraries for scrapping data
import requests

import time
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


#### Step 1: Scrap the data from Wikipedia

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
response

<Response [200]>

In [7]:
#initiate beautifulsoup
soup = BeautifulSoup(response.text, "html.parser")

In [8]:
#get the table with the postcodes into a beautifulsoap object
postalcodestable = soup.select_one('table:nth-of-type(1)')
table_body = postalcodestable.find('tbody')
table_rows = table_body.find_all('tr')

In [9]:
#loop through
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
l

[[],
 ['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n'],
 ['M5A', 'Downtown Toronto', 'Harbourfront\n'],
 ['M5A', 'Downtown Toronto', 'Regent Park\n'],
 ['M6A', 'North York', 'Lawrence Heights\n'],
 ['M6A', 'North York', 'Lawrence Manor\n'],
 ['M7A', "Queen's Park", 'Not assigned\n'],
 ['M8A', 'Not assigned', 'Not assigned\n'],
 ['M9A', 'Etobicoke', 'Islington Avenue\n'],
 ['M1B', 'Scarborough', 'Rouge\n'],
 ['M1B', 'Scarborough', 'Malvern\n'],
 ['M2B', 'Not assigned', 'Not assigned\n'],
 ['M3B', 'North York', 'Don Mills North\n'],
 ['M4B', 'East York', 'Woodbine Gardens\n'],
 ['M4B', 'East York', 'Parkview Hill\n'],
 ['M5B', 'Downtown Toronto', 'Ryerson\n'],
 ['M5B', 'Downtown Toronto', 'Garden District\n'],
 ['M6B', 'North York', 'Glencairn\n'],
 ['M7B', 'Not assigned', 'Not assigned\n'],
 ['M8B', 'Not assigned', 'Not assigned\n'],
 ['M9B', 'Etobicoke', 'Cloverdale

In [77]:
#convert beautifulsoap object to pandas df
df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])

In [78]:
df.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [79]:
#remove trailing /n from neighborhood
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n', '')

In [80]:
#remove rows where boroughs are unassigned
df = df.drop(df[df.Borough == 'Not assigned'].index)

In [81]:
#replace neigborhoods with Not assigned with borough values
df.loc[df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [82]:
#remove first row with None
df = df.drop(df.index[0])

In [21]:
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [87]:
df.shape

(211, 3)

#### Step 3: Add latitude/longitude using geocoder

In [68]:
# get lat/long from csv
latlong_data = pd.read_csv('https://cocl.us/Geospatial_data')
print('data ready')

data ready


In [71]:
latlong_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [84]:
# merge lat/long dataframe with neighborhoods dataframe
updateddf = df.merge(latlong_data, left_on='Postcode', right_on='Postal Code')

In [86]:
updateddf

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
8,M1B,Scarborough,Rouge,M1B,43.806686,-79.194353
9,M1B,Scarborough,Malvern,M1B,43.806686,-79.194353
